In [1]:
import nltk, collections, re, sys, string, pickle
import pandas as pd

In [2]:
def clean_line(line):
    return re.compile('[^a-z]', re.UNICODE).sub('', line.encode('ascii','ignore').lower())

In [3]:
corpus=nltk.corpus.brown.tagged_words(tagset='universal')

In [4]:
num_words = len(corpus)
# words, pos =zip(*corpus[:num_words])
# words, pos

In [5]:
acro_len = 2
ngram = collections.deque(maxlen=acro_len)
pos = collections.deque(maxlen=acro_len)
data_index=0
data = []
while len(ngram)<acro_len:
    cleaned = clean_line(corpus[data_index][0])
    if len(cleaned)>0:
        ngram.append(cleaned)
        pos.append(corpus[data_index][1])
        data.append(cleaned)
    data_index +=1
ngram, pos

(deque(['the', 'fulton']), deque([u'DET', u'NOUN']))

In [24]:
good_pos = [u'ADJ', u'NOUN']
def is_good_acro(pos):
    return reduce(lambda x,y: x and (y in good_pos), pos, True)
#     return True

def make_acro(arr):
    acronym = ''.join(map(lambda x: x[0], arr))
    return acronym


In [25]:
acros = collections.defaultdict(lambda: collections.defaultdict(int))
type(acros), acros

(collections.defaultdict, defaultdict(<function __main__.<lambda>>, {}))

In [26]:
acro_counts = collections.defaultdict(lambda: collections.defaultdict(int))


while data_index<num_words:
    if is_good_acro(pos):
        get_acro = make_acro(ngram)
        acro_counts[get_acro][tuple(ngram)] += 1
    
    cleaned = clean_line(corpus[data_index][0])
    if len(cleaned)>0:
        ngram.append(cleaned)
        pos.append(corpus[data_index][1])
        data.append(cleaned)
    data_index +=1
# acro_counts

In [28]:
min_count = 10
num_counts = 3

acro_dct = {}
for key in acro_counts:
    valid_acros = collections.Counter(acro_counts[key]).most_common(num_counts)
    if len(valid_acros) >= num_counts and valid_acros[-1][1] >= min_count:
        acro_dct[key] = list(zip(*valid_acros)[0])
len(acro_dct), acro_dct

(0, {})

In [10]:
p=data.index('sc')
data[p-3:p+3]

['t', 'simpson', 'greenville', 'sc', 'and', 'three']

In [11]:
cc = collections.Counter(data)

i=0
for acro in acro_dct:
    if acro in cc:
        acro_dct[acro].append((acro,))
        i+=1
        print(acro)
print(i, len(acro_dct))

gt
ga
go
tv
lo
tt
tr
ts
to
lt
th
ti
tb
do
ym
di
ya
yc
da
dw
ys
yt
as
eo
aw
et
ba
rf
ra
wg
wm
bo
bh
bi
wh
wt
ws
bs
bt
ol
oh
of
la
ob
bw
tw
os
op
co
cb
ca
cf
ab
ct
pt
af
pa
sw
ah
pi
po
an
on
hw
al
hi
hl
hm
ha
he
ma
mo
mi
mt
ui
at
aa
ad
ai
is
am
it
ij
im
il
io
in
ie
id
if
ap
nm
no
na
ft
fy
sl
fm
st
si
sh
so
sc
sd
(104, 205)


In [12]:
inverted_acro_dct = {}
for key, acros in acro_dct.items():
    for acro in acros:
        inverted_acro_dct[acro] = key
inverted_acro_dct

{('a', 'big'): 'ab',
 ('a', 'bit'): 'ab',
 ('a', 'certain'): 'ac',
 ('a', 'child'): 'ac',
 ('a', 'couple'): 'ac',
 ('a', 'day'): 'ad',
 ('a', 'drink'): 'ad',
 ('a', 'few'): 'af',
 ('a', 'good'): 'ag',
 ('a', 'great'): 'ag',
 ('a', 'group'): 'ag',
 ('a', 'little'): 'al',
 ('a', 'long'): 'al',
 ('a', 'man'): 'am',
 ('a', 'moment'): 'am',
 ('a', 'new'): 'an',
 ('a', 'number'): 'an',
 ('a', 'part'): 'ap',
 ('a', 'person'): 'ap',
 ('a', 'real'): 'ar',
 ('a', 'result'): 'ar',
 ('a', 'small'): 'as',
 ('a', 'year'): 'ay',
 ('aa',): 'aa',
 ('ab',): 'ab',
 ('ad',): 'ad',
 ('af',): 'af',
 ('ah',): 'ah',
 ('ai',): 'ai',
 ('al',): 'al',
 ('all', 'of'): 'ao',
 ('all', 'right'): 'ar',
 ('am',): 'am',
 ('an',): 'an',
 ('an', 'end'): 'ae',
 ('and', 'a'): 'aa',
 ('and', 'by'): 'ab',
 ('and', 'down'): 'ad',
 ('and', 'even'): 'ae',
 ('and', 'for'): 'af',
 ('and', 'he'): 'ah',
 ('and', 'his'): 'ah',
 ('and', 'i'): 'ai',
 ('and', 'in'): 'ai',
 ('and', 'it'): 'ai',
 ('and', 'more'): 'am',
 ('and', 'of'): 'ao

In [13]:
with open('acros_brown.pickle', 'wb') as handle:
    pickle.dump(acro_dct, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('inverted_acros_brown.pickle', 'wb') as handle:
    pickle.dump(inverted_acro_dct, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
buff = collections.deque(maxlen=acro_len)
replaced = {}
acro_data = []
i=0
for word in data:
    if len(buff) < acro_len:
        buff.append(word)
    else:
        t = tuple(buff)
        if t in inverted_acro_dct:
            acro_data.append(inverted_acro_dct[t])
            buff.clear()
            
            if len(replaced)==0:
                print (i,t)
            replaced[i] = t
        else:
            acro_data.append(buff[0])
        buff.append(word)
    i+=1


(24, ('the', 'jury'))


In [15]:
data[20:26], acro_data[20:26]

(['took', 'place', 'the', 'jury', 'further', 'said'],
 ['took', 'place', 'tj', 'further', 'said', 'in'])

In [16]:
len(data), len(acro_data), len(replaced), replaced

(1005119,
 896353,
 108764,
 {786433: ('on', 'the'),
  786437: ('made', 'a'),
  262151: ('to', 'his'),
  524297: ('in', 'an'),
  873815: ('that', 'he'),
  524301: ('in', 'the'),
  262158: ('but', 'it'),
  524305: ('and', 'he'),
  262162: ('one', 'of'),
  524307: ('may', 'have'),
  262167: ('at', 'the'),
  24: ('the', 'jury'),
  262171: ('that', 'the'),
  786461: ('out', 'of'),
  31: ('that', 'the'),
  262178: ('to', 'the'),
  786467: ('i', 'was'),
  938191: ('a', 'man'),
  961201: ('one', 'of'),
  40: ('of', 'the'),
  524332: ('a', 'new'),
  262191: ('a', 'group'),
  48: ('of', 'the'),
  524339: ('that', 'is'),
  53: ('for', 'the'),
  56: ('in', 'which'),
  972409: ('i', 'was'),
  524351: ('and', 'the'),
  66: ('had', 'been'),
  786499: ('and', 'it'),
  262214: ('to', 'a'),
  655372: ('on', 'the'),
  786507: ('but', 'i'),
  218466: ('with', 'the'),
  786513: ('i', 'was'),
  82: ('in', 'the'),
  288061: ('to', 'be'),
  86: ('which', 'was'),
  524377: ('but', 'the'),
  974155: ('the', 'n

In [17]:
pd.Series(' '.join(data)).to_csv("/home/rishubj/text/brown-data/cleaned_brown_data.csv", index=False)
pd.Series(' '.join(acro_data)).to_csv("/home/rishubj/text/brown-data/cleaned_brown_acro_data.csv", index=False)

/home/rishubj/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
/home/rishubj/.local/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [23]:
with open('replaced.pickle', 'wb') as handle:
    pickle.dump(replaced, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [18]:
# d={key: sum(acro_counts[key].values()) for key in acro_counts
#    if (enough_vals(acro_counts[key].values())>3)}
# len(d), d

In [19]:
# acro_counts['as']

In [20]:
# counts = dict(acro_counts)
# for i in counts:
#     counts[i] = dict(counts[i])
# # counts

In [21]:
# d={key: sum(counts[key].values()) for key in counts
#    if (min(counts[key].values())>=3 and len(counts[key].values())>=2 )}
# d

In [22]:
# b=0
# for i,j in counts.items():
#     for k,l in j.items():
#         if l>4:
#             print (i,k,l)
#             b+=1
# b